# Blcok Recognition

## Import Libraries

In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d
import copy
import pickle

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('./test_imgs/rgb.p', 'rb') as rgb:
    r = pickle.load(rgb)


## Read Test Images

In [3]:
img_dice = r

cv2.imshow(f'color', img_dice)
# cv2.imwrite('../imgs/red_extract.png', each_color_filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Extract Blocks' Masks By Colors

In [4]:
height, width = img_dice.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_dice, cv2.COLOR_BGR2HSV) # cvtdice 함수를 이용하여 hsv 색공간으로 변환

In [5]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [6]:
# RED
lower_red1 = np.array([0, 130, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([160,130,50])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 55, 80])
upper_pink1 = np.array([10, 130, 255])
lower_pink2 = np.array([150,55,80])
upper_pink2 = np.array([179,130,255])

# GREEN
lower_green = (70-20, 50, 50)
upper_green = (70+15, 255, 255)

# YELLOW
lower_yellow = (30-10, 80, 80)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 100, 100)
upper_blue = (100+9, 255, 255)

# VIOLET
lower_violet = (130-20, 60, 60)
upper_violet = (130+20, 255, 255)

In [12]:
domintant_color_area = -1
dominant_color = None

for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)
    each_color_filtered = cv2.bitwise_and(img_dice, img_dice, mask = src_bin)
    
    if color:
        cv2.imshow('src_bin', src_bin)
        # cv2.imwrite('../imgs/red_mask.png', src_bin)
        cv2.imshow(f'{color}_filtered', each_color_filtered)
        # cv2.imwrite('../imgs/red_extract.png', each_color_filtered)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    # Find contours of each color's mask
    contours_color, _ = cv2.findContours(img_mask_color, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Determine the area of each color's contour
    area_color = sum([cv2.contourArea(cnt) for cnt in contours_color])
    
    print(f'{color} area :', area_color)
    
    if area_color > domintant_color_area:
        domintant_color_area = area_color
        dominant_color = color


green area : 76961.0
pink area : 11057.0
yellow area : 24732.5
blue area : 12951.0
violet area : 27184.5
red area : 112168.5


In [9]:
dominant_color

'red'